In [0]:
dbutils.fs.ls('/FileStore/')

Out[14]: [FileInfo(path='dbfs:/FileStore/03__DBUtis___File_System_Utilities.ipynb', name='03__DBUtis___File_System_Utilities.ipynb', size=37614, modificationTime=1708670803000),
 FileInfo(path='dbfs:/FileStore/BadRecords.csv', name='BadRecords.csv', size=177, modificationTime=1704714188000),
 FileInfo(path='dbfs:/FileStore/Returns_data.csv', name='Returns_data.csv', size=31762, modificationTime=1706092817000),
 FileInfo(path='dbfs:/FileStore/SchemaManagementDelta.csv', name='SchemaManagementDelta.csv', size=101806, modificationTime=1709043986000),
 FileInfo(path='dbfs:/FileStore/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1704167189000),
 FileInfo(path='dbfs:/FileStore/_committed_7407023587918272950', name='_committed_7407023587918272950', size=529, modificationTime=1704167189000),
 FileInfo(path='dbfs:/FileStore/_committed_8527669926312473284', name='_committed_8527669926312473284', size=1210, modificationTime=1704167179000),
 FileInfo(path='dbfs:/FileStore/_started_740702358

In [0]:
dbutils.fs.rm('dbfs:/FileStore/streaming/', recurse=True)
#dbutils.fs.rm('dbfs:/FileStore/CheckPoints/', recurse=True)

Out[47]: True

In [0]:
from pyspark.sql.types import *


ReadStream

In [0]:
path1 = 'dbfs:/FileStore/streaming/Countries1.csv'

# To read the Stream , allways prefer schema explicitly for streams 


schema = StructType([
            StructField('Country', StringType()),
            StructField('Citizens', IntegerType())   
])

df = spark.readStream.format('csv').option('header', 'True').schema(schema).load(path1)

In [0]:
#if you absorve the Spark UI , No job is initiated.. Although we are reading a read stream, it is not going to initiate any job.

In [0]:
df.show() # error : Queries with streaming sources must be executed with writeStream.start();

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-677751719813613>:1
----> 1 df.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920     print(self._jdf.showString(n, 20, vertical))
    921 else:


In [0]:
#use display to show error : Option 'basePath' must be a directory
display(df)

In [0]:
## when I'm using the display, it is stating that the base path must be a directory
## The reason it is stating is because I am directly referring to a CSV file while reading as a df
## But streaming data is something it is going to accept the files under this particular directory.
## It cannot just refer to one direct file and it cannot accept the reading data.

In [0]:
source_dir = 'dbfs:/FileStore/streaming/'

In [0]:
df_1 = spark.readStream.format('csv').option('Header', 'True').schema(schema).load(source_dir)

In [0]:
display(df_1)

Country,Citizens
India,10
USA,5
China,10
India,10
Canada,40
Brazil,10
India,5
USA,10
China,5
India,5



>> Spark Streaming is going to invoke a job whenever there is a data available.

>> We are not re executing any of the cell, we are just uploading the file and spark is able to identify that there is a file available and it is going to process it, or it is going to display that.

>> So when you have a streaming source like a folder which have the streaming files getting added when

>> Each file is consider as a micro batch in spark strcutured streaming or Number of files coming as next after one is processing called micro-batch. Ex: If i upload two files for first time then Batch = 1, next time if you upload 3 files Batches = 2 like wise 

>> Click on Cancel Button to Stop the Structure Streaming 


# Spark Streaming Supports different sources and sinks 
Sources 
>> File Source, Table, Kafka Source, Socket Sources, Rate Sources

Sinks
>> File Sink, Kafka Sink, ForeachSink, Console Sink, Table


## Write Stream : To write streaming data to some place 
>> CheckPointLoaction :  checkpointing it is basically used to store the progress of our stream Like having the metadata till where the data is copied.Which means if I am just telling some directory, if there is a stream that is available, it is going to read that particular stream, and it is going to write the data to a destination, and it is going to note down till where the data is been copied.It is not going to store the data, it is just going to have the metadata till where the point is copied.

>> the use of this checkpointing : it is going to give the fault tolerance and resiliency to our streaming data.So to better understand the fault tolerance and the resilient terms, if there is any failure that occurs during the copy of this particular stream, spark is smart enough to start from the point of failure because it is going to store the intermediate metadata in the checkpoints.

>> It will go to the checkpoint location, and it is going to see till where the data is copied and going to begin the data copy from there.

>> It Must be Unique to Each Stream : Means Have a unique checkpointLoaction for each streaming Query , because one streaming query cannot overlap with the other.

In [0]:
dbutils.fs.ls('/FileStore/imgs')

Out[26]: [FileInfo(path='dbfs:/FileStore/imgs/structstreaming1.PNG', name='structstreaming1.PNG', size=107582, modificationTime=1709115587000),
 FileInfo(path='dbfs:/FileStore/imgs/structstreamingO.PNG', name='structstreamingO.PNG', size=141822, modificationTime=1709115587000)]

In [0]:
%sql

create schema if not exists stream

Out[35]: True

In [0]:
 WriteStream = ( df_1.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .queryName('AppendQuery')
        .toTable("stream.AppendTable"))

In [0]:
%sql

select * from stream.AppendTable

Country,Citizens
India,10
USA,5
China,10
India,10
Canada,40
Brazil,10
India,5
USA,10
China,5
India,5


##STOP WRITESTREAM

In [0]:
WriteStream.stop()